## Regularization 

Hillary Cheruiyot 


<hr style="height:2pt">

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

from pandas import datetools

import seaborn as sns

%matplotlib inline

# Continuing Bike Sharing Usage Data

In this project, we will focus on regularization and cross validation. We will continue to build regression models for the [Capital Bikeshare program](https://www.capitalbikeshare.com) in Washington D.C.  See project 3 for more information about the Capital Bikeshare data that we'll be using extensively. 



**Part 1: Data pre-processing**

**1.1** Read in the provided `bikes_student.csv` to a data frame named `bikes_main`. Split it into a training set `bikes_train` and a validation set `bikes_val`. Use `random_state=90`, a test set size of .2, and stratify on month. Remember to specify the data's index column as we read it in.

#### Read the dataset 

*Import the CSV data file into a dataframe*

In [ ]:
bikes_main = pd.read_csv('data/bikes_student.csv', index_col='Unnamed: 0')

*Explore the data*

In [ ]:
bikes_main.columns.values

In [ ]:
bikes_main.shape

In [ ]:
bikes_main.head()

In [ ]:
bikes_main.describe()

#### Describe the dataset

We will work with a dataset contained in data/bikes_student.csv. Each row in this file represents the number of rides by registered users and casual users in a given hour of a specific date in the years 2011 and 2012. There are 36 attributes in total: 


- `Unnamed: 0` (unique row identifier)
- `dteday` (date in the format YYYY-MM-DD, e.g. 2011-01-01)
- `hour` (0 for 12 midnight, 1 for 1:00am, 23 for 11:00pm)
- `year` (0 for 2011, 1 for 2012)
- `holiday` (1 = the day is a holiday, 0 = otherwise)
- `workingday` (1 = the day is a working day, 0 = otherwise)
- `temp` (temperature in Celsius)
- `atemp` (apparent temperature, or relative outdoor temperature, in Celsius)
- `hum` (relative humidity)
- `windspeed` (wind speed)
- `casual` (number of rides that day made by casual riders, not registered in the system)
- `registered` (number of rides that day made by registered riders)
- `count` (total number of rides that day made)
- `Feb` (1 = the day is a Feb day, 0 = otherwise)
- `Mar` (1 = the day is a Mar day, 0 = otherwise)
- `Apr` (1 = the day is a Apr day, 0 = otherwise)
- `May` (1 = the day is a May day, 0 = otherwise)
- `Jun` (1 = the day is a Jun day, 0 = otherwise)
- `Jul` (1 = the day is a Jul day, 0 = otherwise)
- `Aug` (1 = the day is a Aug day, 0 = otherwise)
- `Sep` (1 = the day is a Sep day, 0 = otherwise)
- `Oct` (1 = the day is a Oct day, 0 = otherwise)
- `Nov` (1 = the day is a Nov day, 0 = otherwise)
- `Dec` (1 = the day is a Dec day, 0 = otherwise)
- `spring` (1 = the day is a spring day, 0 = otherwise)
- `summer` (1 = the day is a summer day, 0 = otherwise)
- `fall` (1 = the day is a fall day, 0 = otherwise)
- `Mon` (1 = the day is Monday, 0 = otherwise)
- `Tue` (1 = the day is Tuesday, 0 = otherwise)
- `Wed` (1 = the day is Wednesday, 0 = otherwise)
- `Thu` (1 = the day is Thursday, 0 = otherwise)
- `Fri` (1 = the day is Friday, 0 = otherwise)
- `Sat` (1 = the day is Saturday, 0 = otherwise)
- `Cloudy` (1 = the weather is Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist, 0 = otherwise)
- `Snow` (1 = the weather is Light Snow, Light Rain + Thunderstorm, 0 = otherwise)
- `Storm` (1 = the weather is Heavy Rain + Thunderstorm + Mist, Snow + Fog, 0 = otherwise)
- `month` (1 = Jan, 2= Feb, 3 = Mar, 4 = Apr, 5 = May, 6 = Jun, 7 = Jul, 8 = Aug, 9 = Sep, 10 = Oct, 11 = Nov, 12 = Dec)

If all binary predictors that represent months have the value 0 for a given row, then that row represents an observation done in the month of January.
If all binary predictors that represent week days have the value 0 for a given row, then that row represents an observation made on Sunday.
If all binary predictors that represent season have the value 0 for a given row, then that row represents an observation made on winter.
If all binary predictors that represent weather have the value 0 for a given row, then that row represents an observation made during clear weather.



**Split the dataset**

Split the data (1250 rows) into a training (80%) and a validation set (20%).

In [ ]:
try:
    bikes_train, bikes_val = train_test_split(bikes_main, test_size = 0.2, random_state = 90, stratify=bikes_main['month'])
except:
    # Drop missing lines
    print("Lines with missing values dropped")
    bikes_main = bikes_main.dropna(subset=['month'])
    bikes_train, bikes_val = train_test_split(bikes_main, test_size = 0.2, random_state = 90, stratify=bikes_main['month'])

Verify that each month is represented equally within each set: ca 83 rows per month in the training set and ca. 21 rows per month in the validation set.

In [ ]:
bikes_train.shape

In [ ]:
bikes_train.groupby(['month']).count().dteday

In [ ]:
bikes_val.groupby(['month']).count().dteday

**1.2** As with project 3, the response will be the `counts` column and we'll drop `counts`, `registered` and `casual` for being trivial predictors, drop `workingday` and `month` for being multicolinear with other columns, and `dteday` for being inappropriate for regression. Write code to do this.

Encapsulate this process as a function with appropriate inputs and outputs, and test the code by producing `practice_y_train` and `practice_X_train`


**Drop columns**

Function that takes a data frame and a list of columns as parameters, and return a new dataframe where the columns have been dropped.

In [ ]:
def drop_columns(df, cols):
    for col in cols:
        if col in df:
            df = df.drop(columns = [col])
    return df

Test the function on the train data set and create response and predictor data frames

In [ ]:
cols_to_drop = ['counts', 'registered', 'casual', 'workingday', 'month', 'dteday']
practice_y_train = drop_columns(bikes_train, bikes_train.columns.difference(['counts']))
practice_X_train = drop_columns(bikes_train, cols_to_drop)

Explore the response and predictor data frames ad verify that the specified columns have been correctly dropped

In [ ]:
practice_y_train.head()

In [ ]:
practice_X_train.head()

In [ ]:
practice_X_train.columns.values

**1.3** Write a function to standardize a provided subset of columns in the training/validation/test sets. Remember that while we will be scaling all of the data, we must learn the scaling parameters (mean and SD) from only the training set.

Test the code by building a list of all non-binary columns in the `practice_X_train` and scaling only those columns. Call the result `practice_X_train_scaled`. Display the `.describe()` and verify that we have correctly scaled all columns, including the polynomial columns.


**Standardize function**

Function that takes a data frame and a set of columns as parameters, and scales data using the mean and standard deviation of training data (also a parameter of the function). There are other standardization methods, for example dividing the data by their standard deviation only (without removing the mean). This later method is decribed in the class book. In this homework, we substract the mean as it was done in the lab.

The function will be used to standardize predictors prior to regularization. 

In [ ]:
def standardize_columns(df, cols, scaler):
    df_copy = df.copy()
    df_copy[cols] = scaler.transform(df_copy[cols])
    return df_copy

**Standardize continuous predictors**

After unecessary columns have been dropped, the remaining predictors can be split into two groups.

* binary predictors: 
'holiday', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec', 'spring', 'summer', 'fall', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Cloudy', 'Snow', 'Storm'

* continuous predictors: 
'hour', 'temp', 'atemp', 'hum', 'windspeed', 'year'

In [ ]:
binary_columns = ['holiday','Feb', 'Mar', 'Apr',
       'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec', 'spring',
       'summer', 'fall', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat',
       'Cloudy', 'Snow', 'Storm']
continuous_columns = practice_X_train.columns.difference(binary_columns)
scaler = StandardScaler().fit(practice_X_train[continuous_columns])

practice_X_train_scaled = standardize_columns(practice_X_train.astype(float), continuous_columns, scaler)

Verify if the continuous predictors have mean 0 (almost) and standard deviation 1

In [ ]:
practice_X_train_scaled[continuous_columns].describe()

Verify scaling on validation set

In [ ]:
standardize_columns(bikes_val, continuous_columns, scaler).describe()

**1.4** Write a code to augment the dataset with higher-order features for `temp`, `atemp`, `hum`,`windspeed`, and `hour`. We should include ONLY pure powers of these columns. So with degree=2 we should produce `atemp^2` and `hum^2` but not `atemp*hum` or any other two-feature interactions. 


Encapsulate this process as a function with apropriate inputs and outputs, and test the code by producing `practice_X_train_poly`, a training dataset with qudratic and cubic features built from `practice_X_train_scaled`, and printing `practice_X_train_poly`'s column names and `.head()`.

**Higher-oder function**

Function that takes a dataframe, a list of columns and a polynomial degree, and creates higher order features for the given columns.

For example if the degree is 3 and the columns list is ['x'], then the function will return a new dataframe with new columns x^2 and x^3 additional to the existing x. 

In [ ]:
def create_higher_order(df, cols, degree):
    df_copy = df.copy()
    for col in cols:
        for order in range(2, degree + 1):
            col_name = col + '^' + str(order)
            if not col_name in df:
                df_copy[col_name] = df[col] ** order
    return df_copy

**Create higher order features of continuous variables**

The measured data contains the temperature (temp), relative temperature (atemp), humidity (hum), windspeed and hour.

The data is now augmented with temp^2, temp^3, atemp^2, atemp^3, hum^2, hum^3, windspeed^2, windspeed^3, hour^2, hour^3.

In [ ]:
practice_X_train_poly = create_higher_order(practice_X_train_scaled, ['temp', 'atemp', 'hum', 'windspeed', 'hour'], 3)


Verify the new polynomial features

In [ ]:
new_cols = practice_X_train_poly.columns.difference(list(practice_X_train.columns))
new_cols

In [ ]:
practice_X_train_poly[new_cols].head()

**1.5** Write code to add interaction terms to the model. Specifically, we want interactions between the continuous predictors (`temp`,`atemp`, `hum`,`windspeed`) and the month and weekday dummies (`Feb`, `Mar`...`Dec`, `Mon`, `Tue`, ... `Sat`). That means we SHOULD build `atemp*Feb` and `hum*Mon` and so on, but NOT `Feb*Mar` and NOT `Feb*Tue`. The interaction terms should always be a continuous feature times a month dummy or a continuous feature times a weekday dummy.


Encapsulate this process as a function with appropriate inputs and outputs, and test the code by adding interaction terms to `practice_X_train_poly` and show its column names and `.head()`**


**Interaction terms function**

Function takes a dataframe, a list of continuous predictors, a list of time-related predictors; and returns a new dataframe augmented with interaction terms between each of the continuous predictors and each of the time-related predictors.

In [ ]:
def create_interaction_terms(df, cols_continuous = ['temp', 'atemp', 'hum', 'windspeed'], 
                             cols_time = ['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec', 
                                          'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']):
    df_copy = df.copy()
    for colc in cols_continuous:
        for colb in cols_time:
            col_name = colc + '*' + colb
            if not col_name in df_copy:
                df_copy[col_name] = df_copy[colc] * df_copy[colb]
    return df_copy

**Create interaction terms between countinuous predictors and months/weekday**

In [ ]:
practice_X_train_poly_inter = create_interaction_terms(practice_X_train_poly)

Verify the new interaction terms

In [ ]:
new_cols = practice_X_train_poly_inter.columns.difference(list(practice_X_train_poly.columns))
new_cols

In [ ]:
practice_X_train_poly_inter[new_cols].head()

**1.6** Combine all the code so far into a function that takes in `bikes_train`, `bikes_val`, the names of columns for polynomial, the target column, the columns to be dropped and produces computation-ready design matrices `X_train` and `X_val` and responses `y_train` and `y_val`. The final function should build correct, scaled design matrices with the stated interaction terms and any polynomial degree.

**Function for computation-ready design matrices**

Function that accepts the following parameters:

* a train dataframe
* a validation dataframe
* a polynomial degree
* a list of columns for polynomial features. This list is also the list of continuous predictors.
* the response variable
* a list of columns to remove.

The function performs the following actions:
* retrieves the response variable from the input dataframe
* drops bad columns
* create polynomial terms for continuous predictors up to the given degree
* create interaction terms between continuous predictors and month/weekday binary variables
* standardize all continuous predictors

The function standardizes only the continuous predictors, their corresponding polynomial features and interaction terms are not standardized. Standardizing the predictors would help reduce multicollinearity.

In [ ]:
def get_design_mats(train_df, val_df,  degree, 
                    columns_forpoly=['temp', 'atemp', 'hum','windspeed', 'hour'],
                    target_col='counts', 
                    bad_columns=['counts', 'registered', 'casual', 'workingday', 'month', 'dteday']):
    # retrieve response variables
    y_train = train_df[[target_col]]
    y_val = val_df[[target_col]]
    # drop bad variables
    x_train = drop_columns(train_df, bad_columns)
    x_val = drop_columns(val_df, bad_columns)
    # get binary and continuous columns
    binary_columns = ['holiday','Feb', 'Mar', 'Apr',
       'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec', 'spring',
       'summer', 'fall', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat',
       'Cloudy', 'Snow', 'Storm']
    continuous_columns = x_train.columns.difference(binary_columns)
    # standardize continuous predictors
    scaler = StandardScaler().fit(x_train[continuous_columns])
    x_train = standardize_columns(x_train, continuous_columns, scaler)
    x_val = standardize_columns(x_val, continuous_columns, scaler)
    # create polynomial terms
    x_train = create_higher_order(x_train, columns_forpoly, degree)
    x_val = create_higher_order(x_val, columns_forpoly, degree)
    # create interaction terms
    x_train = create_interaction_terms(x_train)
    x_val = create_interaction_terms(x_val)
    return x_train, y_train, x_val, y_val


**Test the design matrices for train and validation**

In [ ]:
X_train, y_train, X_val, y_val = get_design_mats(bikes_train, bikes_val, 3)

In [ ]:
X_val.describe()

Check the number of rows in the train and validation sets

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

Check the data in the training set

In [ ]:
print(y_train.columns.values)
print(X_train.columns.values)

In [ ]:
y_val.head()

In [ ]:
X_train.describe()

In [ ]:
X_train.head()

Check the data in the validation set

In [ ]:
print(y_val.columns.values)
print(X_val.columns.values)

In [ ]:
y_val.head()

In [ ]:
X_val.describe()

In [ ]:
X_val.head()

## Part 2: Regularization via Ridge

**2.1** For each degree in 1 through 8:

1.  Build the training design matrix and validation design matrix using the function `get_design_mats` with polynomial terms up through the specified degree.

2.  Fit a regression model to the training data.

3.  Report the model's score on the validation data.

**Polynomial models fitting and validation**

For each polynomial degree, we compute the design matrices and fit a polynomial linear regression on the training dataset. We use the model to predict the counts on the validation dataset. The R^2 score is returned not only for the validation dataset but also for the training dataset. We return the coefficients variance and the model with best validation R2 score as well.

The following function performs the above steps for a given max_degree.

In [ ]:
def poly_fit(max_degree = 8):
    r2_scores_val = []
    r2_scores_train = []
    coefs_var = []
    residuals = []
    poly_degrees = range(1,max_degree + 1)
    best_model = None
    best_score_dict = {"degree": 1, "score": 0}
    for degree in poly_degrees:
        # build the design matrices
        X_train, y_train, X_val, y_val = get_design_mats(bikes_train, bikes_val, degree)
        # fit a regression model on training data
        model = LinearRegression().fit(X_train.values, y_train.values)
        # predict on training data
        y_train_predict_ols = model.predict(X_train.values)
        # predict on validation data
        y_val_predict_ols = model.predict(X_val.values)
        # calculate R2 score
        r2_train = r2_score(y_train.values, y_train_predict_ols)
        r2_scores_train.append(r2_train)
        r2_val = r2_score(y_val.values, y_val_predict_ols)
        r2_scores_val.append(r2_val)
        # calculate coefficients variance
        coefs_var.append(np.var(model.coef_))
        # calculate residuals on validation data
        residuals.append(y_val.values - y_val_predict_ols)
        if r2_val > best_score_dict['score']:
            best_score_dict = dict({"degree": degree, "score": r2_val, "predictors": X_train.columns.values})
            best_model = model
    df_scores = pd.DataFrame(dict({"poly_degrees": poly_degrees, 
                              "r2_scores_val": r2_scores_val,
                              "r2_scores_train": r2_scores_train,
                              "coefs_variance": coefs_var,
                              "residuals": residuals}))
    df_scores.poly_degrees = df_scores.poly_degrees.astype('int')
    return df_scores, best_score_dict, best_model

**Reporting scores**

We run the above function for up to 8 degree.

In [ ]:
df_poly_scores, best_poly_score_info, best_poly_model = poly_fit(8)

The model's scores on the validation data is obtained as follows (together with coefficients variance):

In [ ]:
df_poly_scores[['poly_degrees', 'r2_scores_val', 'coefs_variance']]

We now plot how the R2 score and variance change with increasing model complexity (polynomial order).

In [ ]:
def poly_scores_plot(df):
    fig, ax = plt.subplots(1,3, figsize=(18,6))
    ax[0].plot(df.poly_degrees, df.r2_scores_val, ls='-', marker='o', label = 'validation data')
    ax[0].plot(df.poly_degrees, df.r2_scores_train, ls='-', marker='x', label = 'train data')
    ax[0].set_ylabel('$R^2$ score')
    ax[0].set_xlabel('polynomial degree');
    ax[0].set_title('polynomial models\' score')
    ax[0].legend()
    ax[0].grid()
    ax[1].plot(df.poly_degrees, df.coefs_variance, label = 'normalized variance')
    ax[1].set_ylabel('variance')
    ax[1].set_xlabel('polynomial degree');
    ax[1].set_title('polynomial models\' coefficients variance')
    for degree in range(1, len(df.residuals)+1):
        ax[2].violinplot(df.residuals[degree-1], positions=[degree], 
                  showmeans=True, showmedians=True, showextrema=True, vert=False)
    ax[2].set_ylabel('polynomial degree')
    ax[2].set_xlabel('residuals');
    ax[2].set_title('polynomial models\' residuals distribution')
    ax[2].axvline(x=0, color='black', ls='--', alpha=0.4)

In [ ]:
poly_scores_plot(df_poly_scores)

**2.2** Discuss patterns we see in the results from 2.1. Which model would we select, and why?

The R2 score on training data increases as new polynomial features are added to the model (up to 8th power). This trend is expected.

The R2 score on validation data follows a similar trend as on training data; except of a decrease of R2 score when the 4th polynomial order is added. We might need to investigate some multicollinearity effect introduced by those polynomials. By adding a 5th order the increasing R2 trend is resumed.

Most residuals for all models are found below zero for all polynomial orders. 

Normally we would choose the model that provides the best R2 score 0.556 on validation data. Th R2 score indicates the proportion of variance in the counts of bikes riding that can be explained using the predictors. High values of R2 are signs for good model fitness. Therefore the model with 8 degree of polynomials is the one we would select.

However higher model complexity introduces higher variance in the predicted response as shown in the middle plot above. This means that changing the validation data a little bit would come with wider changes in the predicted response. This reduces the significance of regression coefficients and has negative impact on prediction quality. The polynomial degree 6 has the lowest variance ratio with R2 score 0.478 that is still acceptable. We recall having a R2 score 0.445 in project 3 using forward features selection on a quadratic polynomial model. 

If we investigate further by increasing the degree of polynomials, we obtain the plots below. The residual sum of squares becomes higher than the total variance in the original data with polynomial order 17.

At this moment, we would select the model with polynomial terms up to degree 6 because of it seems to offer the most acceptable prediction error, variance and bias. 


In [ ]:
df_poly_scores_18, best_poly_score_info_18, best_poly_model_18 = poly_fit(18)

In [ ]:
print("Best R2 score " + str(best_poly_score_info_18['score']) + " obtained with polynomial degree " + str(best_poly_score_info_18['degree']) + " but at the cost of a large variance " + str(np.var(best_poly_model_18.coef_)))
poly_scores_plot(df_poly_scores_18)

**2.3** Let's try regularizing our models via ridge regression. Build a table showing the validation set $R^2$ of polynomial models with degree from 1-8, regularized at the levels $\lambda = (.01, .05, .1,.5, 1, 5, 10, 50, 100)$. We do not perform cross validation at this point, simply report performance on the single validation set. 


**Model selection with Ridge regularization**

Previously we created different linear regression models with polynomial features. We found out that they provide a good R2 score on validation data. However we were not sure about the importance of predictors and the quality of our estimates in general. Variable selection can be performed by looking for the best subset of predictors, this means searching among 2^193 models. Stepwise forward selection requires less computing effort (18722 possible models). Stepwise backward selection and mixed of both would have a similar computational impact. The best subset of predictors might still miss key predictors, which are significant for the response.

Here we try regularization using a new loss function that augments the MSE with a penalty on large coefficients (except of the intercept). We start with Ridge shrinkage that imposes a constraint on the sum of squares of estimates by shrinking them towards zero. A shrinkage parameter lambda is used to tune the penalty effect. This reduces the variance of those coefficients, what could provide more quality in our predictions and better interpretation of the importance and role of predictors on the response. 

The below function gets a polynomial degree and a list of lambdas as parameters. It creates a ridge regression for each of the lambdas, using predictors which are augmented with their polynomial features up to the specified degree. The function returns the R2 scores obtained on the validation set.

In [ ]:
def ridge_fit(max_degree = 8, lambdas = [0, .01, .05, .1, .5, 1, 5, 10, 50, 100]):
    r2_scores_dict = dict()
    coefs_variance_dict = dict()
    residuals_dict = dict()
    poly_degrees = range(1,max_degree + 1)
    r2_scores_dict['poly_degrees'] = poly_degrees
    coefs_variance_dict['poly_degrees'] = poly_degrees
    residuals_dict['poly_degrees'] = poly_degrees
    best_score_dict = {'degree': 0, 'lambda': 0, 'score': 0}
    best_model = None
    for lamb in lambdas:
        r2_scores_val = [] 
        coefs_variance = []
        residuals = []
        for degree in poly_degrees:
            # build the design matrices
            X_train, y_train, X_val, y_val = get_design_mats(bikes_train, bikes_val, degree)               
            # fit a regularized regression model on training data
            ridge = Ridge(alpha = lamb)
            model = ridge.fit(X_train.values, y_train.values)
            # predict on validation data
            y_val_predict_ols = model.predict(X_val.values) 
            # calculate R2 score   
            r2 = r2_score(y_val.values, y_val_predict_ols)
            r2_scores_val.append(r2)
            # retrieve coefficients
            coefs_variance.append(np.var(model.coef_))
            # calculate residuals
            residuals.append(y_val.values - y_val_predict_ols)
            if (r2 > best_score_dict['score']) & (lamb != 0):
                best_score_dict = {'degree': degree, 'lambda': lamb, 'score': r2, 'predictors': X_train.columns.values}
                best_model = model
        r2_scores_dict[r'$\lambda = ' + str(lamb) + '$'] = r2_scores_val
        coefs_variance_dict[r'$\lambda = ' + str(lamb) + '$'] = coefs_variance
        residuals_dict[r'$\lambda = ' + str(lamb) + '$'] = residuals
    return pd.DataFrame(r2_scores_dict), pd.DataFrame(coefs_variance_dict), pd.DataFrame(residuals_dict), best_score_dict, best_model

In [ ]:
df_ridge_scores, df_ridge_coefs_variance, df_ridge_residuals, best_ridge_score_info, best_ridge_model = ridge_fit()

**Report Ridge R2 scores**

The following table shows the R2 scores obtained when predicting bikes riding with polynomial Ridge regression model on the validation dataset. The models were trained using the training dataset and candidate $\lambda$s.

The R2 scores are displayed for each value of lambda, the Ridge tuning parameter. We added a lambda=0 for comparison purpose which gives the same results as those obtained with ordinary linear regression (without Ridge).

In [ ]:
df_ridge_scores

The following function plots the $R^2$ score on validation data, obtained using a Ridge model for each polynomial degree and lambda combination, as well as the corresponding variance of models coefficients after Ridge schrinkage.

In [ ]:
def ridge_scores_plot(df_scores, df_coefs_variance):
    lambdas = df_scores.columns.difference(['poly_degrees', '$\lambda = 0$']).values
    fig, ax = plt.subplots(1,2, figsize=(15,6)) 
    for lamb in lambdas:
        ax[0].plot(df_scores.poly_degrees, df_scores[lamb], ls='-', marker='o', markersize=3, label = lamb)
    ax[0].set_ylabel('$R^2$ score')
    ax[0].set_xlabel('polynomial degree');
    ax[0].set_title('Ridge polynomial models\' scores on validation data')
    ax[0].legend()
    ax[0].grid()
    for lamb in lambdas:
        ax[1].plot(df_coefs_variance.poly_degrees, df_coefs_variance[lamb], ls='-', marker='o', markersize=3, label = lamb)
    ax[1].set_ylabel('variance')
    ax[1].set_xlabel('polynomial degree');
    ax[1].set_title('polynomial models\' coefficients variance')
    ax[1].legend()
    ax[1].grid()
    

In [ ]:
ridge_scores_plot(df_ridge_scores, df_ridge_coefs_variance)

When higher values of $\lambda$ are used, this penalizes high model complexity, causing models with higher polynomial terms to produce a weak R2 score. 

**2.4** Find the best-scoring degree and regularization combination.

In [ ]:
print("The best-scoring degree appears to be " + str(best_ridge_score_info['degree']) + " with Ridge tuning parameter " + str(best_ridge_score_info['lambda']) + ". The corresponding regression model provides predictions with R2 score = " + str(best_ridge_score_info['score']) + " at the cost of a high variance of coefficients estimates " + str(np.var(best_ridge_model.coef_)))

The variance of coefficients obtained with each degree and lambda combination is given in the table below.

In [ ]:
df_ridge_coefs_variance

**2.5** It's time to see how well our selected model will do on future data. Read in the provided test dataset `data/bikes_test.csv`, do any required formatting, and report the best model's $R^2$ score. How does it compare to the validation set score that made us choose this model? 

**Read test dataset**

In [ ]:
bikes_test = pd.read_csv('data/bikes_test.csv', index_col='Unnamed: 0')

bikes_test.head()

**Create design matrices for test data**

We will perform similar data cleaning and preparation on the test dataset as we did for the training and validation dataset. 
We use the same function get_design_mats for this purposes. 

In [ ]:
X_train_best, y_train_best, X_test_best, y_test_best = get_design_mats(bikes_train, bikes_test, degree = best_ridge_score_info['degree'])

Explore the test dataset

In [ ]:
X_train_best.columns.values

**Predict and get R2 score on test data**

The previous investigations identified the best tuning parameter for the Ridge regularization, combined with the best polynomial order. This model is now fit and used on test data below.

In [ ]:
ridge = Ridge(alpha = best_ridge_score_info['lambda'])
model = ridge.fit(X_train_best.values, y_train_best.values)

In [ ]:
y_test_predict_ridge = model.predict(X_test_best.values)
r2_score_ridge_test = r2_score(y_test_best.values, y_test_predict_ridge)
print("Test score " + str(r2_score_ridge_test))
print("Validation score " + str(best_ridge_score_info['score']))

**2.6** Why do we think our model's test score was quite a bit worse than its validation score? Does the test set simply contain harder examples, or is something else going on?

It is surpising that the Ridge regression with high variance is performing so well on our test data. The model's test score is a bit higher than its validation score. A reason could be that the distribution of test samples is quite similar to those of the validation set. We observed that after scaling the test data using the mean and std learned from the training data, the std of the test data is close to 1 and their mean is almost zero.

It might be interesting to verify why we are doing better on the test set than on the train set. A reason could be that the validation set inculded outliers.

We will perform cross-validation for reducing variability biais, as well as a Lasso regression in order to force some variables to zero and better understand the effect of each variable on the response.

## Part 3: Comparing Ridge, Lasso, and OLS

**3.1** Build a dataset with polynomial degree 1 and fit an OLS model, a Ridge model, and a Lasso model. Use `RidgeCV` and `LassoCV` to select the best regularization level from among `(.1,.5,1,5,10,50,100)`. 

Note: On the lasso model, we will need to increase `max_iter` to 100,000 for the optimization to converge.

**Build a training and validation dataset with polynomial degree 1**

We use the get_design_mats function to create a design matrix.

In [ ]:
X_train_1, y_train_1, X_val_1, y_val_1 = get_design_mats(bikes_train, bikes_val, degree = 1)
X_train_1, y_train_1, X_test_1, y_test_1 = get_design_mats(bikes_train, bikes_test, degree = 1)

**Fit an OLS model**

We fit a linear regression model on the training data without polynomial features of higher order.

In [ ]:
ols_model = LinearRegression().fit(X_train_1.values, y_train_1.values)
y_val_predicted_ols = ols_model.predict(X_val_1.values)
r2_score_val_ols = r2_score(y_val_1.values, y_val_predicted_ols)
y_test_predicted_ols = ols_model.predict(X_test_1.values)
r2_score_test_ols = r2_score(y_test_1.values, y_test_predicted_ols)

**Fit a Ridge model**

In the previous part we performed regularization on a polynomial multiple linear regression model using Ridge cost function  by choosing the tuning parameter and polynomial degree that provided the best $R^2$ score on validation dataset. 

It could however happen by chance that the validation data set was clustered around the regression line of our chosen model. In this case although we got a good validation score, we could experience a much worse score on production. 

By using cross-validation we can select the best tuning parameter with the lowest cross-validation error. Instead of just using one single validation set, we split our data in chunks (folds) on approximately equal size. For each value of the tuning parameter, we use one fold as validation set, and the remaining folds as training set. We average the errors and at the end we chose the $\lambda$ value that provides the lowest mean error. 

For this purpose let's use the built-in, automated cross validation feature of sklearn for Ridge regression. Since our data has a time predictor, we shuffle the data explicitely before searching for the optimal $\lambda$ via cross-validation.

We use a fold size of 20, as recommended by law of thumb in litterature.

In [ ]:
from sklearn.model_selection import KFold
alphas_list = (.1,.5,1,5,10,50,100)
splitter = KFold(n_splits = int(X_train_1.shape[0]/20), random_state=42, shuffle=True)

ridge_cv_model = RidgeCV(alphas=alphas_list, cv=splitter, normalize=False)
ridge_cv_model.fit(X_train_1.values, y_train_1.values)

In [ ]:
print("The Ridge tuning parameter for obtained via cross-validation is " + str(ridge_cv_model.alpha_))

Now let's fit a multilinear regression model with Ridge regularization on the whole training training set.

In [ ]:
ridge_model = Ridge(alpha = ridge_cv_model.alpha_)
ridge_model.fit(X_train_1.values, y_train_1.values)
y_val_predicted_ridge = ridge_model.predict(X_val_1.values)
r2_score_val_ridge = r2_score(y_val_1.values, y_val_predicted_ridge)
y_test_predicted_ridge = ridge_model.predict(X_test_1.values)
r2_score_test_ridge = r2_score(y_test_1.values, y_test_predicted_ridge)

**Fit a Lasso model**

Whereas a Ridge regularization adds a penalty term as sum of sqares of coefficients magnitude, Lasso uses the sum of their absolute values, nulling those with no effect on the response. 

We use cross-validation in order to find the best tuning parameter.

In [ ]:
lasso_cv_model = LassoCV(alphas=alphas_list, cv=splitter, max_iter = 100000)
lasso_cv_model.fit(X_train_1.values, y_train_1.values)

In [ ]:
print("The Lasso tuning parameter for obtained via cross-validation is " + str(lasso_cv_model.alpha_))

Now let's fit a multilinear regression model with Lasso regularization on the whole training training set.

In [ ]:
lasso_model = Lasso(alpha = lasso_cv_model.alpha_)
lasso_model.fit(X_train_1.values, y_train_1.values)
y_val_predicted_lasso = lasso_model.predict(X_val_1.values)
r2_score_val_lasso = r2_score(y_val_1.values, y_val_predicted_lasso)
y_test_predicted_lasso = lasso_model.predict(X_test_1.values)
r2_score_test_lasso = r2_score(y_test_1.values, y_test_predicted_lasso)

The R2 scores of all three approaches are given below.

In [ ]:
pd.DataFrame({'model': ['osl', 'ridge', 'lasso'], 
              'R2 score on val': [r2_score_val_ols, r2_score_val_ridge, r2_score_val_lasso],
              'R2 score on test': [r2_score_test_ols, r2_score_test_ridge, r2_score_test_lasso]})


**3.2** Plot histograms of the coefficients found by each of OLS, ridge, and lasso. What trends do we see in the magnitude of the coefficients?

**Retrieve model coefficients**

We retrieve predictor coefficients from our OLS model

In [ ]:
coefs = pd.DataFrame({'predictors': X_train_1.columns.values, 'ols': ols_model.coef_.ravel()})
coefs.ols = round(coefs.ols.astype('float'), 2)

We retrieve predictor coefficients from our Ridge model

In [ ]:
coefs['ridge'] = ridge_model.coef_.ravel()
coefs.ridge = round(coefs.ridge.astype('float'), 2)

We retrieve predictor coefficients from our Lasso model

In [ ]:
coefs['lasso'] = lasso_model.coef_.ravel()
coefs.lasso = round(coefs.lasso.astype('float'), 2)

Let's have a look at the coefficients

In [ ]:
coefs.head()

**Coefficients histograms**

Below is a distribution plot of all coefficients for each model.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,5))
sns.distplot(coefs.ols, rug=True, kde=False, norm_hist=False, ax=ax[0])
ax[0].set_title('histogram of coefficients - OLS model')
ax[0].set_xlabel('coefficient magnitude')
ax[0].set_ylabel('frequency')
sns.distplot(coefs.ridge, rug=True, kde=False, norm_hist=False, ax=ax[1])
ax[1].set_title('histogram of coefficients - RIDGE model')
ax[1].set_xlabel('coefficient magnitude')
ax[1].set_ylabel('frequency')
sns.distplot(coefs.lasso[coefs.lasso!=0], rug=True, kde=False, norm_hist=False, ax=ax[2])
ax[2].set_title('histogram of coefficients - LASSO model')
ax[2].set_xlabel('coefficient magnitude')
ax[2].set_ylabel('frequency')
plt.show()

The histograms show smaller magnitude for regularized coefficients. With the OLS model, coefficients are distributed in a wider values range. Ridge and Lasso models have shrinked coefficients with less variance. Lasso caused a lot of coefficients to be zero (we did not show them in the Lasso histogram in order to keep the remaining distribution visible, the list of zero coefficients is given below). 

Outliers coefficient are particularly large in the OLS model. Their magnitude is significantly reduced by Ridge and set to zero by Lasso. Most coefficients are found around zero.

The overall variance of the coefficients for each model is given below.

In [ ]:
np.var(coefs.ols), np.var(coefs.ridge), np.var(coefs.lasso)

The following coefficients were removed by Lasso regression. The corresponding predictors or interaction terms have very little or no effect on the number of bike rides.

In [ ]:
coefs[coefs.lasso==0].predictors.values

**3.3** The plots above show the overall distribution of coefficient values in each model, but do not show how each model treats individual coefficients. Build a plot which cleanly presents, for each feature in the data, 1) The coefficient assigned by OLS, 2) the coefficient assigned by ridge, and 3) the coefficient assigned by lasso.

We first sort the coefficients by overall magnitude.

In [ ]:
coefs['all'] = abs(coefs.ols) + abs(coefs.ridge) + abs(coefs.lasso)
coefs = coefs.sort_values(by = ['all'])

The stacked barplot below shows the magnitude of the coefficients associated with the predictors. The predictors with the cummulative largest coefficient magnitude are displayed at the beginning of the plot. The bar shows the magnitude obtained with each of the three models (OLS - blue, Ridge - orange, Lasso - green). 

Bars oriented to the left identify those predictors which have a negative effect on the predicted count of ridership. The remaining ones have a positive effect. A step increase of the later would imply a step increase in the count by the magnitude of the coefficient.

Lasso provides the best interpretability. For example humidity has a considerable impact on ridership during certain months (Sept). High temperature brings definetely more ridership. It is strange that Storm has no effect on the ridership. This is probably because of the lack of data during stormy weather.


In [ ]:
coefs.plot.barh(x='predictors', y=['ols', 'ridge', 'lasso'], stacked=True, legend=True, figsize=(10, 30))
plt.axvline(x=0, color='black', alpha=0.4)
plt.xlabel('magnitude')
plt.ylabel('variables')
plt.title('coefficients estimates by OLS - Ridge - Lasso')
plt.show()

**3.4** What trends do we see in the plot above? How do the three approaches handle the correlated pair `temp` and `atemp`?

When the coefficient is large, the sign of coefficients appears to be the same regardless the approach used. The coefficients which were set to zero by Lasso tend to be handled differently by Ridge and OLS when it comes to their sign. 

OLS handles temp as a positive predictor of ridership, and atemp as a negative predictore of ridership. This is strange because both predictors are positively correlated. We would expect temp and atemp to behave in the same direction. This is not the case for Lasso and Ridge approaches, where both temp and atemp predict an increase in ridership.

Ridge assumes that both temp and atemp would have approximately the same impact on the response. Lasso however seems to have detected the collinearity between both and tried to shrink atemp towards zero.

In [ ]:
coefs[(coefs.predictors=='temp')|(coefs.predictors=='atemp')].plot.barh(x='predictors', y=['ols', 'ridge', 'lasso'], stacked=True, legend=True, figsize=(6, 4))

## Part 4: Reflection

**4.1** Reflect back on the `get_design_mats` function we built. Writing this function useful in the analysis? What issues might we have encountered if we copy/pasted the model-building code instead of tying it together in a function? Does a `get_design_mat` function seem wise in general, or are there better options?

The get_design_mats function was useful for better code visibility, debugging, it promoted less lines of code, allowing to reuse the same code logic through the questions. If the data structure is the same throughout the project, then using such a function is wise for exploring various polynomial and regularized models. An improvement of get_design_mats could be adding a flag for enabling/disabling the creation of interaction terms, and also providing the list of predictors that could interact as a parameter to the function. get_design_mats could also handle exceptions.

**4.2** What are the costs and benefits of applying ridge/lasso regularization to an overfit OLS model, versus setting a specific degree of polynomial or forward selecting features for the model?

Adding polynomials in OLS improves the validation score while increasing prediction variance and risk of overfitting; Ridge/Lasso reduces the variance. Polynomial terms bring multicolinearity issues with them, what negatively affects interpretability; by shrinking the coefficients of less important predictors, Ridge/Lasso helps avoiding that issue. Forward selection identifies the subset of predictors which give the less error, but this approach can miss important predictors. Lasso can help in the special case of high dimensionality where there are more predictors than observations. The down side is, Ridge/Lasso comes at the cost of computation power especcially when the data set is very big.  

**4.3** This project posed a purely predictive goal: forecast ridership as accurately as possible. How important is interpretability in this context? Considering, e.g., the lasso and ridge models from part 3, how would we react if the models predicted well, but the coefficient values didn't make sense once interpreted?

Although prediction was more important than interpretability in this context, having for example two strange coefficients for temp and atemp, or storm should also be considered closely, as they could affect prediction on another test dataset. Inference should be ideally investigated even if the main goal is prediction. 


**4.4** Reflect back on our original goal of helping BikeShare predict what demand will be like in the week ahead, and thus how many bikes they can bring in for maintenance. In the view, did we accomplish this goal? If yes, which model would we put into production and why? If not, which model came closest, what other analyses might we conduct, and how likely do we think they are to work

For any given month, day of week, weather forecast and hour of the day, our models are able to predict the number of rides that will be made by both registered and casual users with an acceptable R2 score (see summary table below). However without a proper analysis of coefficients variance, bias, residuals and coefficients p-values/CI, we think that the prediction goal was not fully achieved. 

We suggest a model obtained with interaction terms, polynomial model with degree 6 regularized with Lasso using a tuning parameter obtained via cross-validation. This model would predict with R2 score around 0.478, would bring low coefficients variance, and would consider only significant predictors.  

In [ ]:
pd.DataFrame({"Model": ['original', 'polynomial degree 2', 'polynomial degree 2 with forward selection', 'polynomial degree 6 with int. terms', 'multilinear polynomia degree 8 with inter. terms', 'polynomial degree 8 with Ridge lamba 0.1', 'polynomial degree 1 with inter. terms', 'polynomial degree 1 with inter. terms and Ridge', 'polynomial degree 1 with inter. terms and Lasso'], "Test R2": [0.406, 0.496, 0.445, 0.478, 0.556, 0.556, 0.349, 0.388, 0.383]})